In [4]:
import os
from openai import OpenAI

client = OpenAI(
    # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx"
    api_key=os.getenv("BAILIAN_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

completion = client.chat.completions.create(
    # 模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
    model="qwen-plus",
    messages=[
        {"role": "user", "content": "以json格式输出一个json字符串"},
    ],
    response_format={"type": "json_object"},
)
print(completion.choices[0].message.content)

{
  "json字符串": "{\\\"name\\\": \\\"张三\\\", \\\"age\\\": 30, \\\"city\\\": \\\"北京\\\"}"
}


In [5]:
print(completion.choices[0].message.content)

{
  "json字符串": "{\\\"name\\\": \\\"张三\\\", \\\"age\\\": 30, \\\"city\\\": \\\"北京\\\"}"
}


In [1]:
from config import rag_cfg

for item in rag_cfg["docling"]:
    print(item)

do_picture_description
do_ocr
do_table_structure
do_cell_matching


In [1]:
from src.prompt import get_prompt

prompt = get_prompt("query_route", "rag")

print(prompt)

<Role>
You are a query routing assistant that determines which collections are most relevant to the user's question.
</Role>
<Instructions>
1. Carefully read user's Question.
2. For each collection in **Collections**, assign a relevance score from 0 to 5:
   - 0 = Completely irrelevant
   - 1 = Very low relevance
   - 2 = Low relevance
   - 3 = Moderate relevance
   - 4 = High relevance
   - 5 = Very high relevance
3. Base your scoring only on the semantic meaning of the question and the collection name.
4. Output your answer strictly in **valid JSON format**, where:
   - Keys are the collection names (exactly as given)
   - Values are the relevance scores (integers from 0 to 5)
</Instructions>
<Example>
**Input Question**:
卷积网络和 Transformer 网络的区别是什么？

**Collections**:

{{
   {{
      "collection": "Resnet",
      "keywords": [...],
   }},
   {{
      "collection": "ViT",
      "keywords": [...]
   }},
   {{
      "collection": "Knowledge Distillation",
      "keywords": [...]
   }}
}}

In [37]:
def split_messages(messages: list[dict[str, str]], max_len: int = 2):
    result = []

    for msg in messages:
        role = msg.get("role", "")
        content = msg.get("content", "")

        # 按 max_len 切分 content
        parts = [content[i : i + max_len] for i in range(0, len(content), max_len)]

        # 为每段创建一条新消息
        for part in parts:
            result.append({"Header 1": role, "content": part})

    return result


messages = [
    {"role": "system", "content": "系统消息……"},
    {"role": "user", "content": "用户消息非常长，非常长，非常长……（>300）"},
    {"role": "assistant", "content": "助手回复"},
]

processed = split_messages(messages, max_len=10)

for item in processed:
    print(item)

{'Header 1': 'system', 'content': '系统消息……'}
{'Header 1': 'user', 'content': '用户消息非常长，非常'}
{'Header 1': 'user', 'content': '长，非常长……（>3'}
{'Header 1': 'user', 'content': '00）'}
{'Header 1': 'assistant', 'content': '助手回复'}


In [ ]:
d = {"text1": 1, "text2": 2, "text3": 3, "text4": 4, "text5": 5}
a = list(map(int, d.values()))
a

In [ ]:
from config import rag_cfg

a = rag_cfg.get("query_route")

if a:
    print("Query route is set.")

In [ ]:
from RAG.retrieve_pipeline.utils import extract_title_from_markdown

text = """
下面用**直观版 → 技术版 → 数学版 → 示例流程图**四层深度，系统地给你讲清楚 **RLHF（Reinforcement Learning from Human Feedback）原理**。
（考虑到你本身做 KD、元学习水印等，我会尽量给出“机制级”解释。）

---

# 🌟 一、直观版：RLHF 到底在做什么？

RLHF的任务是：

> **让模型不仅“会回答”（SFT），还“回答得更接近人类偏好”。**

它怎么做到？

1. **人类给模型输出打分/排序 → 得到偏好数据**
2. **训练一个奖励模型（Reward Model）来模仿这些偏好**
3. **用强化学习，让大模型在生成时最大化奖励模型的得分，同时保持语言流畅**

它的核心思想：

> **用 RM 当作“可微奖励函数”，用 RL 让策略模型符合这个奖励函数。**

---

# 🌟 二、技术版：RLHF 三阶段原理（ChatGPT 使用的流程）

## ✔️ **阶段 1：SFT（监督微调）**

用高质量 `(prompt, answer)` 数据训练出一个初始模型 π₀。
它只是“能回答问题”，但不懂对齐，不懂安全规范。

**输出结果：SFT 模型 = 好学生，但不懂“老师喜欢什么”。**

---

## ✔️ **阶段 2：奖励模型 RM（Preference Model）**

输入：模型生成的两个回答 `y_pos` 和 `y_neg`
标签：人类说 “y_pos 比 y_neg 更好”

训练目标：

> 让 RM 输出：
> reward(y_pos) > reward(y_neg)

具体形式：

[
\mathcal{L}_{RM} = - \log\sigma(r(y^{+}) - r(y^{-}))
]

训练出的 RM 是一个 **评分器**，可以对任意回答给分。

**输出结果：奖励模型 RM = 模仿人类偏好的评分系统。**

---

## ✔️ **阶段 3：强化学习（PPO）优化策略模型 RLHF-PPO**

现在有两个模型：

* **策略模型 π**（你最终要的 LLM）
* **奖励模型 r**（你让它成为“老师”）

目标：

> **让策略模型生成的答案获得高奖励分数 r(answer)，但又不能偏离原本语言能力太远。**

所以目标函数是：

[
\max_{\pi} \ \mathbb{E}[ r(y) ] - \beta \cdot KL(\pi ,||, \pi_0)
]

其中 KL 约束是为了：

* 防止 reward hacking
* 保持语言质量
* 避免模型偏离过头（崩坏）

### ⭐ PPO 的作用

PPO 是一种“稳定强化学习优化器”，它的技巧：

* 限制新旧策略概率比 `ratio` 不要变化太大（clip 技术）
* 避免训练发散
* 让 KL 不会突然爆炸

简化版本的 PPO 目标：

[
\mathcal{L}_{PPO} = \mathbb{E}\left[\min( ratio \cdot A, \ clip(ratio,1-\epsilon,1+\epsilon) \cdot A )\right]
]

其中 advantage A 用 RM 算：

[
A = r(y) - \beta KL(\pi||\pi_0)
]

---

# 🌟 三、数学原理（核心概念用一句话总结）

| 组件           | 数学本质                                                  | 用一句话讲            |
| ------------ | ----------------------------------------------------- | ---------------- |
| **奖励模型 RM**  | pairwise Bradley-Terry / logistic preference learning | 学“人类更喜欢哪一个回答”    |
| **策略优化 PPO** | KL-regularized policy gradient                        | “让模型追求高分但不能偏离太多” |
| **KL 项**     | 软约束防退化                                                | 保持语言质量稳定         |
| **RLHF 整体**  | KL-正则化的最大期望奖励                                         | 本质是优化“奖励 + 稳定性”  |

核心优化问题：

[
\pi^* = \arg\max_\pi \ \mathbb{E}_{y\sim\pi}[r(y)] - \beta KL(\pi||\pi_0)
]

---

# 🌟 四、RLHF 的优势与问题

## 👍 优势

* 强大的可控性
* 能实现“安全”、“礼貌”、“拒答”等行为
* 能学到人类的偏好，而不是死记硬背答案

## 👎 问题（DPO 就是为解决这些问题）

* PPO 难训练
* RM 会“奖励作弊”（reward hacking）
* 训练成本高（大批量生成 + RL）
* 不稳定（KL 控制很敏感）

DPO 就是在数学上重新推导 RLHF 的目标，使得：

* 不需要 RM
* 不需要 PPO
* 直接最大化偏好对数似然
* 更稳定

---

# 🌟 五、RLHF 整体流程图（超清晰）

```
             ┌──────────────────────────────────┐
             │    Stage 1: SFT 监督微调         │
             │   用人工写的好答案训练 π0        │
             └──────────────────────────────────┘
                            │
                            ▼
             ┌──────────────────────────────────┐
             │  Stage 2: 奖励模型 RM             │
             │  输入：人类偏好(优 vs 劣)         │
             │  学习：r(y+) > r(y-)              │
             └──────────────────────────────────┘
                            │
                            ▼
             ┌──────────────────────────────────┐
             │  Stage 3: RLHF-PPO 强化学习       │
             │  策略模型 π 生成答案              │
             │  RM 打分 → Advantage             │
             │  PPO 更新 π，保持 KL 稳定        │
             └──────────────────────────────────┘
                            │
                            ▼
                      「最终模型 π*」
            = 更符合人类偏好的大语言模型
```

---

如果你愿意，我还能继续讲：

✅ RLHF vs DPO 数学推导对比
✅ RM 的具体网络结构（GPT decoder head + scalar head）
✅ PPO 在 LLM 中的 trick（KL 控制、length norm、reward shaping）
✅ RLHF 在蒸馏水印项目中的作用
（尤其是你做的元学习蒸馏，是可以和 RLHF 的 KL 正则产生类比的）

你想继续哪个部分？

"""


title = extract_title_from_markdown(text)
for line in title:
    print(line)

In [ ]:
import sys
import os

# Add the src directory to the Python path
sys.path.append(os.path.abspath("src"))

from src.llm.factory import get_llm
from src.llm.base import add_user_message

llm = get_llm(llm_provider="deepseek", model="deepseek-chat")

messages = []
add_user_message(messages, prompt)
response = llm.chat(messages, format="json")

print(response)

In [ ]:
import json

data = json.loads(response)
print(data.get("false"))

In [ ]:
json_str = """
{
    "false": "No title found",
}
"""
data = json.loads(json_str)
print(data.get("false"))